# Homework 2. Collaborative Filtering News RecSys : CS6365 Fall 2025

Justin Mittereder - G49843234

In [57]:
import pandas as pd
import datetime

col_names  = ['Impression ID', 'User ID', 'Time', 'History', 'Impressions']
train_behaviors_df = pd.read_csv('data/MINDsmall_train/behaviors.tsv', sep='\t', header=None, names=col_names)
train_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
train_behaviors_df.reset_index(drop=True, inplace=True)
val_behaviors_df = pd.read_csv('data/MINDsmall_dev/behaviors.tsv', sep='\t', header=None, names=col_names)
val_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
val_behaviors_df.reset_index(drop=True, inplace=True)

col_names  = ['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
train_news_df = pd.read_csv('data/MINDsmall_train/news.tsv', sep='\t', header=None, names=col_names)
val_news_df = pd.read_csv('data/MINDsmall_dev/news.tsv', sep='\t', header=None, names=col_names)


In [58]:
train_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions
0,107691,U100,11/12/2019 7:34:12 AM,N20121 N33998 N45954 N55743 N50095 N18870 N534...,N61235-0 N54489-0 N42597-0 N7800-1 N61408-0 N2...
1,86767,U1000,11/13/2019 11:16:18 PM,N29641 N1789 N41244,N35273-0 N7618-0 N55281-0 N4021-0 N16148-0 N38...
2,74345,U1000,11/14/2019 10:37:21 PM,N29641 N1789 N41244,N14478-0 N58264-0 N7494-0 N46917-0 N22257-0 N1...
3,46640,U1000,11/14/2019 1:00:56 AM,N29641 N1789 N41244,N16148-0 N43628-0 N13094-0 N32774-0 N57034-0 N...
4,155923,U10001,11/11/2019 5:30:21 AM,N47937 N51706 N56447 N61319 N27644 N18030 N272...,N35729-0 N56598-0 N48759-0 N49685-0 N33632-0 N...


In [59]:
train_news_df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [60]:
train_behaviors_df["unix_timestamp"] = pd.to_datetime(
    train_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

val_behaviors_df["unix_timestamp"] = pd.to_datetime(
    val_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

earliest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].min())
latest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].max())
earliest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].min())
latest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].max())

print(f"Training impressions are from between {earliest_train_time} and {latest_train_time} . ")
print(f"Validation impressions are from between {earliest_val_time} and {latest_val_time} . ")


Training impressions are from between 2019-11-08 19:00:19 and 2019-11-14 18:59:13 . 
Validation impressions are from between 2019-11-14 19:00:01 and 2019-11-15 18:58:03 . 


In [61]:
#get full df of all user impressions
import datetime
import pandas as pd

def get_impressions(df, include_history=False):
    rows = []
    for idx, row in df.iterrows():
        user_id = row["User ID"]
        time = row["unix_timestamp"]
        impressions = row["Impressions"]

        # split impressions
        for impression in impressions.split():
            article, click = impression.split("-")
            click = int(click)

            rows.append({
                "user_id": user_id,
                "timestamp": time,
                "article": article,
                "impression": click
            })

        if(include_history):
            # split impressions
            history = str(row['History'])
            for article in history.split(" "):
                rows.append({
                    "user_id": user_id,
                    "timestamp": "N/A",
                    "article": article,
                    "impression": 1 
                })
        
    impressions_df = pd.DataFrame(rows)
    impressions_df.sort_values(by=['user_id', 'timestamp'], inplace=True)
    impressions_df.reset_index(drop=True, inplace=True)
    return impressions_df    


impressions_df = get_impressions(train_behaviors_df)
impressions_df = impressions_df[impressions_df['impression'] == 1]
print("Only Considering Clicked on Articles")
print("Train: Number of Impressions: ", len(impressions_df))
print("Train: Number of articles: ", impressions_df['article'].nunique())
print("Train: Number of Users: ", impressions_df['user_id'].nunique())
print(impressions_df.head())

val_impressions_df = get_impressions(val_behaviors_df)
val_impressions_df = val_impressions_df[val_impressions_df['impression'] == 1]
print("Only Considering Clicked on Articles")
print("Val: Number of Impressions: ", len(val_impressions_df))
print("Val: Number of articles: ", val_impressions_df['article'].nunique())
print("Val: Number of Users: ", val_impressions_df['user_id'].nunique())
print(val_impressions_df.head())

Only Considering Clicked on Articles
Train: Number of Impressions:  236344
Train: Number of articles:  7713
Train: Number of Users:  50000
    user_id   timestamp article  impression
3      U100  1573544052   N7800           1
134   U1000  1573686978  N53875           1
166   U1000  1573693256  N29739           1
180   U1000  1573693256   N7670           1
307   U1000  1573771041  N58656           1
Only Considering Clicked on Articles
Val: Number of Impressions:  111383
Val: Number of articles:  2212
Val: Number of Users:  50000
   user_id   timestamp article  impression
1       U1  1573802905  N20036           1
13     U10  1573798095  N32536           1
59  U10000  1573807178  N31958           1
63  U10000  1573811650  N50775           1
93  U10000  1573823809  N60215           1


Get Most Popular items based on number of successful impressions in training set

In [77]:
success_impressions_df = impressions_df[impressions_df['impression'] == 1]
print("num successful impressions: ", len(success_impressions_df))
print("unique articles that have been clicked on: ", success_impressions_df['article'].nunique())

# Count impressions per article
impression_counts = (success_impressions_df.groupby("article").size().reset_index(name="impression_count"))
impression_counts.sort_values('impression_count', ascending=False, inplace=True)
#print("Impressions Article: \n", impression_counts)

# Sort by most clicked
popular_items = impression_counts.sort_values("impression_count", ascending=False)
#print(popular_items)

# Convert to top 1000 list (most clicked first)
pop_list = popular_items["article"].head(1000).tolist()
#print(pop_list)

num successful impressions:  236344
unique articles that have been clicked on:  7713


Map all User IDs and Article IDs to integers so that we can use them as row and columns in user-article matrix


In [62]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
article_encoder = LabelEncoder()

impressions_df['user_idx'] = user_encoder.fit_transform(impressions_df['user_id']) #Map all User IDs to integers so we can use them for the rows in the user-article matrix 
impressions_df['article_idx'] = article_encoder.fit_transform(impressions_df['article']) #Map all Article IDs to integers so we can use them for the columns in the user-article matrix 

Map all Article Ids in popularity list to integers using article encoder

In [78]:
pop_idx = article_encoder.fit_transform(pop_list)

In [63]:
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

num_users = impressions_df['user_idx'].nunique()
num_articles = impressions_df['article_idx'].nunique()

user_article_matrix = csr_matrix((np.ones(len(impressions_df)),(impressions_df['user_idx'], impressions_df['article_idx'])),shape=(num_users, num_articles))
print("User Article Matrix Shape: ", user_article_matrix.shape)

#calculate the similarity between articles based on which users have interacted with which articles using cosine similarity function on transpose(user-article) matrix -> article-user matrix
article_article_sim = cosine_similarity(user_article_matrix.T) #items are similar if they've been clicekd on by similar users/have similar user impression vectors

User Article Matrix Shape:  (50000, 7713)


To recommend items for a user:
- Look at the items the user clicked
- Gather similar items from item_item_sim
- Rank by similarity score
- Remove already-clicked items
- Take Top-K

In [64]:
val_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions,unix_timestamp
0,8158,U1,11/15/2019 7:28:25 AM,N23571 N58267 N25682 N10646 N32607 N57737 N523...,N14637-0 N20036-1,1573802905
1,71327,U10,11/15/2019 6:08:15 AM,N27612 N36699 N64777 N9120 N9803 N57967 N2945,N33397-0 N46917-0 N11930-0 N58612-0 N47612-0 N...,1573798095
2,5144,U10000,11/15/2019 1:16:49 PM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N48740-0 N51470-0 N1952-0 N23675-0 N56969-0 N6...,1573823809
3,41696,U10000,11/15/2019 8:39:38 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N29393-0 N20036-0 N30290-0 N31958-1 N23513-0 N...,1573807178
4,1307,U10000,11/15/2019 9:54:10 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N50775-1 N31958-0 N53572-0 N5472-0 N58251-0 N3...,1573811650


In [65]:
#split articles previously clicked by users in validation set
val_behaviors_df["History_list"] = val_behaviors_df["History"].apply(lambda x: x.split() if isinstance(x, str) else [])

#take all impressions in validation set that user actually clicked on
def get_clicked_articles(impressions_str):
    clicks = []
    for imp in impressions_str.split():
        article, clicked = imp.split("-")
        if int(clicked) == 1:
            clicks.append(article)
    return clicks

#create new col in val set called clicked_articles with all articles user viewed
val_behaviors_df['clicked_articles'] = val_behaviors_df['Impressions'].apply(get_clicked_articles)

In [66]:
# Get sets of users
train_users = set(train_behaviors_df['User ID'].unique())
val_users = set(val_behaviors_df['User ID'].unique())

# Cold-start users: in validation but not in training
cold_start_users = val_users - train_users

# Number of cold-start users
num_cold_start = len(cold_start_users)
print("Number of cold-start users:", num_cold_start)

print("Number of Users in Train set: ", train_behaviors_df['User ID'].nunique())
print("Number of Users in Val set: ", train_behaviors_df['User ID'].nunique())
all_users = train_users.union(val_users)
num_total_unique_users = len(all_users)

print("Total unique users across train and validation:", num_total_unique_users)

Number of cold-start users: 44057
Number of Users in Train set:  50000
Number of Users in Val set:  50000
Total unique users across train and validation: 94057


In [68]:
#map User Ids in validation set to user ids from user_encoder used by train set
def user_transform(user_series, encoder):
    # try transforming; if unseen, return -1
    user_array = user_series.values
    user_idx = np.full(len(user_array), -1, dtype=int)
    mask = np.isin(user_array, encoder.classes_)
    known_users = user_array[mask]
    user_idx[mask] = encoder.transform(known_users)
    return user_idx

val_behaviors_df['user_idx'] = user_transform(val_behaviors_df['User ID'], user_encoder)


In [72]:
import pandas as pd
import numpy as np

# Flatten history lists
all_articles = [(i, article) for i, lst in enumerate(val_behaviors_df['History_list']) for article in lst]
flat_df = pd.DataFrame(all_articles, columns=['row_idx', 'article'])

# Map known articles to indices
mask = np.isin(flat_df['article'], article_encoder.classes_)
flat_df['item_idx'] = -1
flat_df.loc[mask, 'item_idx'] = article_encoder.transform(flat_df.loc[mask, 'article'])

# Group back into lists per row
history_idx_grouped = flat_df.groupby('row_idx')['item_idx'].apply(list)

val_behaviors_df['history_idx'] = val_behaviors_df.index.map(history_idx_grouped)

# Flatten clicked_articles lists
clicked_all_articles = [(i, article) for i, lst in enumerate(val_behaviors_df['clicked_articles']) for article in lst]
clicked_flat_df = pd.DataFrame(clicked_all_articles, columns=['row_idx', 'article'])

# Map known articles to indices
clicked_mask = np.isin(clicked_flat_df['article'], article_encoder.classes_)
clicked_flat_df['item_idx'] = -1
clicked_flat_df.loc[clicked_mask, 'item_idx'] = article_encoder.transform(clicked_flat_df.loc[clicked_mask, 'article'])

# Group back into lists per row
clicked_idx_grouped = clicked_flat_df.groupby('row_idx')['item_idx'].apply(list)

val_behaviors_df['clicked_idx'] = val_behaviors_df.index.map(clicked_idx_grouped)


In [73]:
val_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions,unix_timestamp,History_list,clicked_articles,user_idx,history_idx,clicked_idx
0,8158,U1,11/15/2019 7:28:25 AM,N23571 N58267 N25682 N10646 N32607 N57737 N523...,N14637-0 N20036-1,1573802905,"[N23571, N58267, N25682, N10646, N32607, N5773...",[N20036],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 426, -1, -1]",[1251]
1,71327,U10,11/15/2019 6:08:15 AM,N27612 N36699 N64777 N9120 N9803 N57967 N2945,N33397-0 N46917-0 N11930-0 N58612-0 N47612-0 N...,1573798095,"[N27612, N36699, N64777, N9120, N9803, N57967,...",[N32536],-1,"[-1, -1, -1, -1, -1, -1, -1]",[-1]
2,5144,U10000,11/15/2019 1:16:49 PM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N48740-0 N51470-0 N1952-0 N23675-0 N56969-0 N6...,1573823809,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N60215],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[-1]
3,41696,U10000,11/15/2019 8:39:38 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N29393-0 N20036-0 N30290-0 N31958-1 N23513-0 N...,1573807178,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N31958],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[2820]
4,1307,U10000,11/15/2019 9:54:10 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N50775-1 N31958-0 N53572-0 N5472-0 N58251-0 N3...,1573811650,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N50775],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[-1]


In [ ]:
#val_behaviors_df.to_csv('data/MINDsmall_dev/val_behaviors_transformed.csv')

In [75]:
val_behaviors_df['history_idx']

0        [-1, -1, -1, -1, -1, -1, -1, -1, -1, 426, -1, -1]
1                             [-1, -1, -1, -1, -1, -1, -1]
2        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
3        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
4        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
                               ...                        
73147                                 [-1, -1, -1, -1, -1]
73148                                 [-1, -1, -1, -1, -1]
73149    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
73150    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
73151    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
Name: history_idx, Length: 73152, dtype: object

In [82]:
import numpy as np

def recommend_from_history(history_idx, article_article_sim, pop_idx, k):
    """
    history_idx: list of article indices the user has clicked previously
    item_item_sim: item-item similarity matrix from training
    K: number of items to recommend
    """
    if len(history_idx) == 0:
        return pop_idx[:k]  #cold-start user so recommend popular
    
    # Remove -1 (unseen articles)
    history_idx = [i for i in history_idx if i != -1]
    if len(history_idx) == 0:
        return pop_idx[k:] #all unseen articles in history, cold-start user so recommend popular
    
    # Sum similarity scores over all history items
    scores = np.sum(article_article_sim[history_idx, :], axis=0)

    # Remove articles already clicked
    scores[history_idx] = -np.inf

    # Get top-K items
    topk_idx = np.argpartition(-scores, k)[:k]          # fast Top-K
    topk_sorted = topk_idx[np.argsort(-scores[topk_idx])]  # sort top-K
    return topk_sorted.tolist()

# Replace NaN or non-list entries with empty lists
val_behaviors_df['history_idx'] = val_behaviors_df['history_idx'].apply(
    lambda x: x if isinstance(x, list) else []
)

# Apply function to each row
val_behaviors_df['recommended_idx_10'] = val_behaviors_df['history_idx'].apply(
    lambda x: recommend_from_history(x, article_article_sim, pop_idx, 10)
)

val_behaviors_df['recommended_idx_50'] = val_behaviors_df['history_idx'].apply(
    lambda x: recommend_from_history(x, article_article_sim, pop_idx, 50)
)



In [83]:
val_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions,unix_timestamp,History_list,clicked_articles,user_idx,history_idx,clicked_idx,recommended_idx,recommended_idx_10,recommended_idx_50
0,8158,U1,11/15/2019 7:28:25 AM,N23571 N58267 N25682 N10646 N32607 N57737 N523...,N14637-0 N20036-1,1573802905,"[N23571, N58267, N25682, N10646, N32607, N5773...",[N20036],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 426, -1, -1]",[1251],"[185, 671, 5143, 5139, 5140, 5141, 5142, 5137,...","[185, 671, 5143, 5139, 5140, 5141, 5142, 5137,...","[185, 671, 5150, 5149, 5123, 5151, 5167, 5166,..."
1,71327,U10,11/15/2019 6:08:15 AM,N27612 N36699 N64777 N9120 N9803 N57967 N2945,N33397-0 N46917-0 N11930-0 N58612-0 N47612-0 N...,1573798095,"[N27612, N36699, N64777, N9120, N9803, N57967,...",[N32536],-1,"[-1, -1, -1, -1, -1, -1, -1]",[-1],"[816, 890, 480, 532, 548, 511, 607, 775, 252, ...","[816, 890, 480, 532, 548, 511, 607, 775, 252, ...","[745, 443, 401, 561, 259, 214, 67, 486, 54, 95..."
2,5144,U10000,11/15/2019 1:16:49 PM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N48740-0 N51470-0 N1952-0 N23675-0 N56969-0 N6...,1573823809,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N60215],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[-1],"[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 4372, 2970, 678..."
3,41696,U10000,11/15/2019 8:39:38 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N29393-0 N20036-0 N30290-0 N31958-1 N23513-0 N...,1573807178,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N31958],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[2820],"[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 4372, 2970, 678..."
4,1307,U10000,11/15/2019 9:54:10 AM,N10059 N46978 N53234 N3345 N3345 N9155 N9653 N...,N50775-1 N31958-0 N53572-0 N5472-0 N58251-0 N3...,1573811650,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",[N50775],-1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",[-1],"[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 2970, 4372, 678...","[3186, 4930, 3308, 6865, 1176, 4372, 2970, 678..."


In [85]:
def hit_rate_at_k(df, k):
    hits = 0
    total = 0
    
    rec_col = f'recommended_idx_{k}'

    for _, row in df.iterrows():
        recs = set(row[rec_col])
        true_clicked = set([i for i in row['clicked_idx'] if i != -1])  # ignore articles not seen in training because these couldn't have been recommended in the first place
        
        if len(true_clicked) == 0:
            continue  # skip if no ground truth clicked items
        
        # Hit if at least one recommended item is in clicked items
        hit = False
        for article in true_clicked:
            if article in recs:
                hit = True
                break
        
        if hit:
            hits+=1
        total += 1
    
    return (hits / total) * 100

val_hr10 = hit_rate_at_k(val_behaviors_df, 10)
val_hr50 = hit_rate_at_k(val_behaviors_df, 50)
print(f"Validation Hit Rate@10: {val_hr10}")
print(f"Validation Hit Rate@50: {val_hr50}")

Validation Hit Rate@10: 2.011983134847252
Validation Hit Rate@50: 2.7270261607120845
